In [2]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...


In [3]:
cd yolov5

C:\Users\mahen\Intranet\Modele_IA\yolov5


In [4]:
!pip install -r requirements.txt

  Obtaining dependency information for gitpython>=3.1.30 from https://files.pythonhosted.org/packages/e9/bd/cc3a402a6439c15c3d4294333e13042b915bbeab54edc457c723931fed3f/GitPython-3.1.43-py3-none-any.whl.metadata
  Obtaining dependency information for pillow>=10.3.0 from https://files.pythonhosted.org/packages/dc/83/1470c220a4ff06cd75fc609068f6605e567ea51df70557555c2ab6516b2c/pillow-11.0.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for requests>=2.32.2 from https://files.pythonhosted.org/packages/f9/9b/335f9764261e915ed497fcdeb11df5dfd6f7bf257d4a6a2a686d80da4d54/requests-2.32.3-py3-none-any.whl.metadata
  Obtaining dependency information for thop>=0.1.1 from https://files.pythonhosted.org/packages/bb/0f/72beeab4ff5221dc47127c80f8834b4bcd0cb36f6ba91c0b1d04a1233403/thop-0.1.1.post2209072238-py3-none-any.whl.metadata
  Obtaining dependency information for torch>=1.8.0 from https://files.pythonhosted.org/packages/78/18/7a2e56e2dc45a433dea9e1bf46a65e234294c9c470ccb

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
conda-repo-cli 1.0.75 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.75 requires PyYAML==6.0.1, but you have pyyaml 6.0 which is incompatible.
conda-repo-cli 1.0.75 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


   -------------------- ----------------- 111.4/203.1 MB 281.6 kB/s eta 0:05:26
   -------------------- ----------------- 111.4/203.1 MB 282.3 kB/s eta 0:05:25
   -------------------- ----------------- 111.5/203.1 MB 283.5 kB/s eta 0:05:24
   -------------------- ----------------- 111.5/203.1 MB 283.3 kB/s eta 0:05:24
   -------------------- ----------------- 111.5/203.1 MB 283.8 kB/s eta 0:05:23
   -------------------- ----------------- 111.6/203.1 MB 286.1 kB/s eta 0:05:21
   -------------------- ----------------- 111.6/203.1 MB 286.4 kB/s eta 0:05:20
   -------------------- ----------------- 111.6/203.1 MB 286.5 kB/s eta 0:05:20
   -------------------- ----------------- 111.6/203.1 MB 287.2 kB/s eta 0:05:19
   -------------------- ----------------- 111.7/203.1 MB 284.3 kB/s eta 0:05:22
   -------------------- ----------------- 111.7/203.1 MB 284.3 kB/s eta 0:05:22
   -------------------- ----------------- 111.7/203.1 MB 284.3 kB/s eta 0:05:22
   -------------------- ----------------

In [1]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models, transforms
from torch.autograd import Variable

In [2]:
def load_inceptionv3_model():
    model = models.inception_v3(pretrained=True)
    model.eval()  # Mettre le modèle en mode évaluation
    return model

In [19]:
def load_image(image_path):
    # Charger l'image avec OpenCV
    image = cv2.imread(image_path)
    return image

In [3]:
def load_yolo_model():
    # Charger le modèle YOLOv5 small pré-entraîné
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
    return model

In [6]:
def detect_faces_yolo(model, image):
    results = model(image)
    detections = results.xyxy[0].numpy()  # Extraire les boîtes englobantes
    return detections

In [7]:
def detect_eyes_opencv(face_region):
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    gray_face = cv2.cvtColor(face_region, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray_face, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return eyes

In [6]:
def detect_eyes_opencv(face_region):
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    gray_face = cv2.cvtColor(face_region, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray_face, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return eyes

In [8]:
def align_face(image, face_box, eyes):
    x1, y1, x2, y2 = map(int, face_box)
    face_region = image[y1:y2, x1:x2]
    
    if len(eyes) >= 2:
        eye_1, eye_2 = eyes[:2]
        eye_1_center = (eye_1[0] + eye_1[2] // 2, eye_1[1] + eye_1[3] // 2)
        eye_2_center = (eye_2[0] + eye_2[2] // 2, eye_2[1] + eye_2[3] // 2)

        # Calculer l'angle entre les yeux
        if eye_1_center[0] > eye_2_center[0]:
            left_eye, right_eye = eye_2_center, eye_1_center
        else:
            left_eye, right_eye = eye_1_center, eye_2_center
        
        delta_x = right_eye[0] - left_eye[0]
        delta_y = right_eye[1] - left_eye[1]
        angle = np.arctan2(delta_y, delta_x) * 180 / np.pi

        # Aligner le visage en utilisant la rotation
        eyes_center = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)
        rotation_matrix = cv2.getRotationMatrix2D(eyes_center, angle, scale=1)
        aligned_face = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))
        return aligned_face
    else:
        print("Moins de deux yeux détectés, alignement non effectué.")
        return face_region

In [9]:
def preprocess_inceptionv3(face_region):
    preprocess = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    face_tensor = preprocess(face_region).unsqueeze(0)  # Ajouter une dimension batch
    return face_tensor

In [10]:
def extract_embedding_inceptionv3(model, face_region):
    face_tensor = preprocess_inceptionv3(face_region)
    with torch.no_grad():
        embedding = model(face_tensor)
    return embedding

In [12]:
def draw_rectangle_and_eyes(image, x1, y1, x2, y2, eyes=None):
    # Dessiner un rectangle autour du visage
    cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Couleur verte (0, 255, 0)
    
    # Si les yeux sont détectés, dessiner des points sur les yeux
    if eyes is not None:
        for (ex, ey, ew, eh) in eyes:
            # Ajuster les coordonnées des yeux pour l'image globale
            adjusted_eye_x = int(ex + x1)
            adjusted_eye_y = int(ey + y1)
            cv2.circle(image, (adjusted_eye_x, adjusted_eye_y), radius=10, color=(0, 0, 255), thickness=-1)  # Couleur rouge pour les yeux

    return image

In [13]:
def display_image(image):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

In [27]:
def process_image_yolo_inceptionv3(image_path):
    image = load_image(image_path)
    
    # Charger les modèles
    yolo_model = load_yolo_model()
    inceptionv3_model = load_inceptionv3_model()

    # Détecter les visages avec YOLOv5
    detections = detect_faces_yolo(yolo_model, image)
    
    if len(detections) == 0:
        print("Aucun visage détecté.")
        return
    
    # Pour chaque visage détecté
    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        
        # Filtrer les détections pour ne garder que les visages
        if int(cls) == 0 and conf > 0.5:  # Classe 0 correspond à "personne"
            print(f"Visage détecté avec confiance {conf}")
            face_region = image[int(y1):int(y2), int(x1):int(x2)]
            
            # Détecter et aligner le visage
            eyes = detect_eyes_opencv(face_region)
            aligned_face = align_face(image, (x1, y1, x2, y2), eyes)
            
            # Extraire l'embedding avec InceptionV3
            embedding = extract_embedding_inceptionv3(inceptionv3_model, aligned_face)
            print(f"Embedding généré: {embedding}")
            
            # Afficher l'image du visage aligné
            display_image(aligned_face)
        else:
            print("Aucun visage valide détecté.")

In [21]:
image = "C:/Users/mahen/Intranet/Modele_IA/images/profil_picture/selena_profile_picture.jpg"

In [22]:
images = "C:/Users/mahen/Intranet/Modele_IA/images/picture/selena_picture_1.jpg"

In [28]:
process_image_yolo_inceptionv3(image)

Using cache found in C:\Users\mahen/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-20 Python-3.11.5 torch-2.2.2+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Visage détecté avec confiance 0.9084011316299438
Moins de deux yeux détectés, alignement non effectué.
Embedding généré: tensor([[ 1.89462e-01, -6.33493e-01, -5.99313e-01, -1.00432e+00, -3.57196e-01,  8.31120e-01, -1.43864e-01, -1.61972e+00, -1.79009e+00, -1.16951e+00, -1.05932e-01, -1.58925e+00, -2.32428e+00, -2.87505e-01,  5.47722e-01, -1.82523e-01, -4.81537e-01, -3.35399e-01, -2.14787e-01, -4.24318e-01,  7.94485e-01, -5.84164e-01,
         -9.61458e-01, -1.45852e+00, -1.02480e+00,  9.24446e-01,  4.86233e-01, -1.46401e-01,  8.99955e-01,  5.37848e-02, -4.29221e-01,  1.84188e-01,  5.12259e-01, -4.46297e-01,  3.86084e-01, -5.44121e-01,  6.43739e-01, -1.42914e-01,  1.13595e+00,  2.34162e-01,  1.62188e-01,  2.41549e-01,  1.21156e+00,  2.12611e+00,
          1.73797e-01,  1.56738e-01,  4.89944e-02,  1.20158e+00, -1.94620e-01,  4.98237e-01,  1.45865e+00, -2.78620e-01, -8.73171e-01,  1.73619e-01, -1.21639e-01, -1.24890e+00, -9.81721e-01,  6.25911e-01, -1.69002e+00, -1.15990e+00, -1.77893e+00

Aucun visage valide détecté.


In [29]:
process_image_yolo_inceptionv3(images)

Using cache found in C:\Users\mahen/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-20 Python-3.11.5 torch-2.2.2+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Visage détecté avec confiance 0.8587438464164734
Moins de deux yeux détectés, alignement non effectué.
Embedding généré: tensor([[-8.95049e-01, -2.38070e-01, -6.27754e-01, -1.24384e+00, -1.51436e+00, -4.09224e-01, -1.56994e+00, -1.39641e+00, -1.08157e+00, -2.80866e+00, -1.41416e-01, -1.79607e-01, -1.57537e+00, -1.52723e+00, -6.52140e-01, -1.09938e+00,  1.16653e-01, -8.54053e-01, -1.55303e+00,  4.19816e-02,  3.22728e-01, -3.56824e-01,
          5.54227e-02,  1.97983e-01, -8.30023e-01, -6.45473e-01, -8.90722e-01, -1.01107e+00, -1.27002e+00, -2.16930e-01,  5.51343e-01, -1.39447e+00, -6.36696e-01, -1.15363e+00, -3.68618e-01,  3.73216e-01,  1.25331e+00, -3.11080e-01, -3.32910e-01, -5.00444e-03, -1.24415e+00, -4.76721e-01, -1.28832e+00, -4.13852e-01,
         -1.62386e+00, -1.51656e-01, -7.01994e-01, -5.38739e-01,  5.92770e-01,  6.04251e-02, -7.80451e-01,  2.07620e-01, -9.95328e-01,  2.56385e-01,  2.97425e-01, -1.08973e-01, -1.03083e-01,  1.40819e-01,  1.24669e+00, -1.82875e+00,  7.70902e-01

Visage détecté avec confiance 0.7990054488182068
Moins de deux yeux détectés, alignement non effectué.
Embedding généré: tensor([[ 2.08846e+00,  3.01040e-01, -3.31074e-01, -4.70535e-02,  5.74105e-01, -3.09010e-01, -3.88565e-01, -1.90443e+00, -1.11315e+00, -1.32816e+00,  2.50104e-01, -1.37807e+00, -9.25813e-01, -7.64295e-01, -7.86642e-01, -1.66969e-01, -5.89899e-01, -2.55161e-01, -1.32995e+00,  1.17538e-01, -2.84059e-01, -9.60979e-01,
         -2.09686e+00, -2.05977e+00, -5.76132e-01, -1.16011e+00, -2.31477e-01, -1.18257e+00, -1.69061e+00, -1.09482e+00, -1.06959e+00, -1.69845e+00, -4.11828e-01,  1.13226e+00,  1.41394e+00,  2.98455e-01,  8.06829e-01, -4.03894e-01, -1.03406e+00, -1.75043e-01,  7.37520e-02, -4.95657e-01, -8.75665e-01, -7.85927e-02,
         -1.23472e+00, -9.97065e-01, -4.10459e-01, -9.63569e-01, -8.49365e-01, -1.41248e-02,  5.60119e-01, -2.17341e-01, -3.00622e-03, -5.05450e-01, -3.57567e-01, -2.47753e-01, -1.09770e+00, -6.19614e-01,  1.93590e-01,  9.67998e-01, -2.17653e-01

Visage détecté avec confiance 0.7366359233856201
Moins de deux yeux détectés, alignement non effectué.
Embedding généré: tensor([[-1.18485e+00, -9.35695e-01, -6.91086e-01, -1.19542e+00, -9.66567e-01, -1.69325e-01, -1.60564e+00,  1.44596e+00, -7.80883e-01, -1.45944e+00, -2.28043e-01, -5.74277e-01, -4.79560e-01, -6.71019e-01,  6.93693e-01, -1.18532e+00, -1.14278e+00, -2.00314e-01, -1.01814e+00,  1.23653e-01, -9.27713e-01, -1.89373e-01,
         -8.25242e-01, -2.00507e+00, -9.70578e-02, -2.00326e+00, -8.31625e-01, -3.49135e-01, -7.44648e-01, -1.78589e-01, -1.07558e-01, -1.39875e+00, -5.41302e-01, -4.22720e-01, -6.36184e-01, -6.49101e-01, -1.08811e+00, -9.11369e-01, -9.71037e-01,  4.32449e-01, -1.80836e+00, -1.25468e+00,  4.77101e-01,  3.42926e-01,
         -1.09011e+00, -2.29002e-01, -5.35755e-01, -1.15464e+00,  2.23412e-02, -6.92972e-01, -1.15027e+00, -3.71420e-01, -8.71287e-01, -1.20173e+00, -6.62908e-01, -1.30700e+00, -9.47612e-01, -9.33605e-01, -1.00196e+00, -3.88162e-01, -3.71143e-01

Visage détecté avec confiance 0.7346781492233276
Moins de deux yeux détectés, alignement non effectué.
Embedding généré: tensor([[-2.96315e-02, -2.11965e-01, -1.20272e+00, -6.81361e-01,  6.99345e-01,  1.05819e-02, -5.29022e-01, -2.94550e-01,  1.03216e-01, -8.97216e-01, -1.29649e-01, -3.68600e-01, -1.02324e+00, -1.03535e+00, -1.24503e+00,  2.57542e-02,  2.74648e-02, -1.13831e+00, -1.20726e+00, -6.61867e-01, -7.04638e-01,  1.12254e+00,
         -5.95838e-01,  9.82006e-02, -1.83862e-01, -1.66517e-01, -4.04343e-01, -4.43982e-01, -8.82670e-01, -4.65267e-01, -1.14032e+00, -4.72523e-01, -9.71556e-02,  3.00505e-01,  1.56178e-01, -8.75112e-01,  2.41778e-01, -5.81398e-01,  3.38298e-01, -2.82656e-01, -7.75848e-01, -2.45807e-01, -8.01958e-01, -2.85625e-01,
         -8.96230e-01, -8.20419e-01, -3.10517e-01, -7.87867e-02, -2.97278e-01,  1.97104e-01, -9.46244e-01, -6.87275e-02,  4.39590e-01, -6.52587e-03,  3.42299e-01,  4.19346e-01, -1.76576e-01, -3.28217e-01,  5.89067e-02,  6.19813e-01,  4.33690e-01

Visage détecté avec confiance 0.7028934955596924
Moins de deux yeux détectés, alignement non effectué.
Embedding généré: tensor([[ 2.25641e+00,  1.63649e+00,  2.52362e-01,  6.63577e-01,  8.82198e-01,  3.74107e+00,  6.31356e-01, -6.45950e-01, -1.13963e+00, -1.09488e-01, -2.19988e-01, -1.34325e+00, -6.32329e-01, -1.35357e-01,  8.31951e-02, -1.60806e+00, -4.22729e-02, -7.78746e-01, -7.24941e-02, -4.53920e-01, -7.15513e-01, -6.48419e-01,
         -9.43722e-01,  3.30994e-01, -8.93819e-01, -6.86421e-01,  1.22364e+00, -1.10801e+00,  1.25624e+00,  3.34450e+00, -4.02544e-01, -3.66827e-01,  2.65789e-01, -7.32271e-01, -5.78815e-01,  2.48091e-01,  6.52337e-03, -6.11685e-01,  3.08513e-01, -1.65727e+00,  4.35556e-01, -9.73872e-01, -3.38513e-01,  7.22544e-01,
         -1.29894e+00, -1.52558e+00, -1.57169e-01,  6.40123e-01, -7.21452e-01, -1.51788e+00, -1.15585e+00,  1.49571e+00, -1.56867e-01,  4.27097e-01, -1.61181e+00, -1.02059e+00, -9.51648e-01, -2.06245e+00, -3.90168e-01,  7.56513e-01, -5.71049e-01

Visage détecté avec confiance 0.7026879787445068
Moins de deux yeux détectés, alignement non effectué.
Embedding généré: tensor([[-6.49432e-01, -4.60418e-01,  1.15762e+00, -7.94038e-01, -1.30476e+00, -4.82255e-01, -1.20911e+00, -2.63774e-03,  3.36174e-01, -1.61748e+00, -6.49857e-02, -3.43766e-02, -6.70869e-01, -5.86947e-01,  5.24960e-01, -5.78443e-01, -8.72656e-01,  1.17265e-01, -5.34740e-01,  4.56622e-01,  6.11060e-01, -7.10638e-01,
          4.69374e-01, -6.26658e-01, -2.96580e-01, -2.66129e-01, -7.48283e-01, -5.46028e-01, -4.13050e-01,  7.45380e-01,  8.94016e-01, -5.81505e-01, -1.73580e-01, -1.06325e+00, -4.29015e-01,  1.13370e-01, -5.87505e-01,  8.29830e-02, -1.01473e+00, -3.04610e-01, -1.05153e+00, -5.07573e-01, -4.93254e-01,  3.82814e-01,
         -1.16715e+00, -2.37558e-01, -2.95839e-01, -5.89441e-01,  2.68911e-01, -5.34783e-01, -7.69888e-01,  5.14448e-03, -8.84278e-01, -2.70624e-01, -5.15670e-01, -5.39882e-02, -4.83081e-01,  1.19906e+00,  8.87030e-01, -5.27640e-01,  2.88183e-01

Visage détecté avec confiance 0.5949143767356873
Moins de deux yeux détectés, alignement non effectué.
Embedding généré: tensor([[-6.28587e-01,  4.20899e-01,  2.60151e-01,  6.77491e-01, -6.53197e-02,  3.62119e-01, -1.45738e+00,  3.57692e+00, -1.03631e+00, -9.91239e-01, -1.11596e+00, -2.07052e-01, -9.33933e-01, -2.29576e-01,  2.12141e+00, -1.22678e+00, -1.37009e+00,  9.99332e-01,  3.68732e-01, -2.89345e-03, -5.24312e-01, -3.05383e-01,
         -3.98295e-02, -1.01589e+00, -3.75966e-01, -1.24111e+00, -3.02784e-01,  3.31415e-01,  6.22381e-02,  1.29978e+00, -1.44198e+00, -7.19646e-01, -1.95105e-01, -4.68614e-01, -3.69226e-01, -8.48624e-01, -6.29545e-01, -1.12159e+00,  3.21786e-02,  2.47158e-01, -2.30067e-01,  1.65023e-01,  9.02435e-01,  1.56885e-01,
         -4.67944e-02, -8.10597e-01,  2.90333e-02, -1.01598e+00, -4.20094e-01, -3.94660e-01, -1.01933e+00, -7.20336e-01, -3.60823e-01, -9.36177e-01, -6.44438e-01, -9.15559e-01, -4.27465e-01, -5.72651e-01, -1.04192e+00,  1.67248e-01, -6.57515e-01

Visage détecté avec confiance 0.5698919892311096
Moins de deux yeux détectés, alignement non effectué.
Embedding généré: tensor([[ 3.92739e+00,  3.38814e-01,  1.58834e+00, -1.26162e+00,  1.29267e-01,  2.85645e+00,  6.47482e-01,  1.03313e+00, -1.27201e-01, -2.95655e-01, -1.04443e+00, -6.82330e-01, -3.01297e-01, -1.66252e+00, -2.37962e-01, -6.14776e-01, -5.43559e-01, -1.48189e-01, -1.07917e+00, -1.81351e+00, -1.04074e+00,  3.01290e-01,
         -1.10469e+00,  1.70824e+00,  2.67435e-01, -1.57426e+00, -1.33858e+00, -2.32830e+00, -1.49652e-01, -1.21910e-01, -2.75082e-02, -1.39267e+00, -1.75573e+00, -8.06442e-01,  6.35193e-01,  2.12126e-01,  1.35766e+00, -3.12021e-01, -1.29394e+00, -9.14578e-01, -1.09159e+00, -8.80915e-01,  1.83391e+00,  1.10390e+00,
         -1.77756e+00, -9.27298e-01, -1.02725e+00, -3.28113e-01,  3.60112e-01, -1.37992e+00, -4.38843e-01,  2.20101e+00, -1.01031e+00, -9.47659e-01, -1.03653e+00, -9.26941e-01, -1.58761e+00, -1.59250e+00, -5.37812e-01,  8.49476e-02, -8.08194e-01

Aucun visage valide détecté.


In [33]:
import torch
print(torch.__version__)

2.5.0+cpu


In [26]:
def display_image(image):
    cv2.imshow("Image avec détection de visages", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()